In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import date

pd.set_option('display.max.columns',100)

##sources of file 
dfnpath = 'Excel/15101160_20220519_0443.xlsx'
dfopath = 'Record/Data.csv'
catnpath = 'Excel/DiscriptionCategories.xlsx'
catopath = 'Record/DiscriptionCategories.xlsx'

## 
df = pd.read_excel(dfnpath,index_col=[0])
dfo = pd.read_csv(dfopath,encoding='latin1',index_col=[0])
catn = pd.read_excel(catnpath)
cato = pd.read_excel(catopath)
df = df.dropna(axis=1,how='all')
dfo = dfo.dropna(axis=1,how='all')


In [ ]:

## delete columns from the table 
df = df.drop(columns=['Account Number','Balance','Transaction Type','Sort Code'])
catn = catn.drop(columns=['Transaction Date','Debit Amount','Credit Amount'])

# combine new and old cat table
cat = pd.concat([catn,cato], ignore_index=True)
df = pd.concat([df,dfo], ignore_index=True)


## change data type into string 
df['Transaction Description'] = df['Transaction Description'].astype(str)

In [ ]:
## merge cat table with df 
MergeTable = pd.merge(df,
                        cat,
                        on = 'Transaction Description',
                        how = 'left',
                        suffixes = ('','_DROP')).filter(regex='^(?!.*_DROP)')

MergeTable = MergeTable.drop_duplicates(ignore_index=True)

##identifly Nan cat and save

catn = MergeTable[MergeTable['Categories'].isna()]
catn = catn.drop_duplicates(ignore_index=True)
catn = catn.loc[:, ~catn.columns.str.contains('^Unnamed')]
catn.to_excel(catnpath,index=False)
catn.head()

In [ ]:

cato = cat[cat['Categories'].notna()]
cato = cato.drop_duplicates(ignore_index=True)
cato = cato.loc[:, ~cato.columns.str.contains('^Unnamed')]
cato.to_excel(catopath,index=False)
cato.head()

MergeTable.groupby(['Categories'],as_index = True).agg({'Debit Amount':'sum','Credit Amount':'sum'}).plot(kind='pie',y='Debit Amount',title = 'Total Expenses',startangle=60)

In [ ]:
##store the new Categories to excel file 
MergeTable = MergeTable.loc[:, ~MergeTable.columns.str.contains('^Unnamed')]
MergeTable.to_csv(dfopath, index=False)

# move and rename dfn to backup file 

backup_path = 'Record/Backup/'+str(date.today())+'.csv'
os.rename(dfnpath,backup_path)
